# RNN Serial by Location

This version of the RNN runs the model on each location separately, one at a time. Two main runs:
1. Run separate model at each location
2. Run same model with multiple fitting calls at different locations, compare prediction accuracy for all of them at the end.

In [ ]:
import numpy as np
import pandas as pd
from utils import print_dict_summary, print_first, str2time, logging_setup
import pickle
import matplotlib.pyplot as plt
import logging
import os.path as osp
from moisture_rnn_pkl import pkl2train
from moisture_rnn import create_rnn_data2 
from utils import hash2
from moisture_rnn import RNN
import reproducibility
from data_funcs import rmse

In [ ]:
logging_setup()

In [ ]:
file_names=["reproducibility_dict2.pickle",'test_NW_202401.pkl','test_CA_202401.pkl']
file_dir='data'
file_paths = [osp.join(file_dir,file_name) for file_name in file_names]

In [ ]:
# read/write control
train_file='train.pkl'
train_create=False   # if false, read
train_write=False
train_read=True

In [ ]:
if train_create:
    logging.info('creating the training cases from files %s',file_paths)
    # osp.join works on windows too, joins paths using \ or /
    train = pkl2train(file_paths)
if train_write:
    with open(train_file, 'wb') as file:
        logging.info('Writing the rain cases into file %s',train_file)
        pickle.dump(train, file)
if train_read:
    logging.info('Reading the train cases from file %s',train_file)
    with open(train_file,'rb') as file:
        train=pickle.load(file)

In [ ]:
import yaml

with open("params.yaml") as file:
    params_all = yaml.safe_load(file)
params_all.keys()

In [ ]:
# from module_param_sets import param_sets

In [ ]:
param_sets_keys=['rnn_repro', 'rnn']
# cases=[list(train.keys())[0]]
cases=list(train.keys())[205:210]
# cases.remove('reproducibility')
cases

In [ ]:
logging.info('Running over parameter sets %s',param_sets_keys)
logging.info('Running over cases %s',cases)

## Separate Models by Location

In [ ]:
rmses = []
for k in param_sets_keys:
    params = params_all[k]
    print("~"*80)
    print("Running with params:")
    print(params)
    if k == "rnn_repro":
        case = 'reproducibility'
        logging.info('Running reproducibility case')
        rnn_dat = create_rnn_data2(train[case], params)
        reproducibility.set_seed()
        rnn = RNN(params)
        m, errs = rnn.run_model(rnn_dat)
        print(f"RMSE: {errs}")
    else:
        for case in cases:
            # Increase Val Frac so no errors, TODO fix validation
            params.update({
                'train_frac': .5,
                'val_frac': .2
            })
            print("~"*50)
            logging.info('Processing case %s',case)
            print_dict_summary(train[case])
            # Format data & Run Model
            rnn_dat = create_rnn_data2(train[case], params)
            reproducibility.set_seed()
            rnn = RNN(params)
            m, errs = rnn.run_model(rnn_dat)
            # Add model output to case
            train[case]['m']=m
            # Get RMSE Prediction Error
            rmses.append(errs['prediction'])
            print(f"RMSE: {errs}")


In [ ]:
pd.DataFrame({'Case': cases, 'RMSE': rmses}).style.hide(axis="index")

In [ ]:
logging.info('test-plk2train.ipynb done')

In [ ]:
import importlib
import moisture_rnn
importlib.reload(moisture_rnn)
from moisture_rnn import RNN

## Training at Multiple Locations

Still sequential

In [ ]:
params = params_all['rnn']
params.update({'epochs': 20})
rnn_dat = create_rnn_data2(train['reproducibility'], params)

In [ ]:
reproducibility.set_seed()
rnn = RNN(params)

In [ ]:
for k in param_sets_keys:
    params = params_all[k]
    print("~"*80)
    print("Running with params:")
    print(params)
    if k == "rnn_repro":
        continue
    else:
        for case in cases:
            # Increase Val Frac so no errors, TODO fix validation
            params.update({
                'train_frac': .5,
                'val_frac': .2
            })
            print("~"*50)
            logging.info('Processing case %s',case)
            print_dict_summary(train[case])
            rnn_dat = create_rnn_data2(train[case], params)
            rnn.fit(rnn_dat['X_train'], rnn_dat['y_train'],
                   validation_data=(rnn_dat['X_val'], rnn_dat['y_val']))
            # run_rnn_pkl(train[case],param_sets[i])

### Predict 

In [ ]:
# Predict Cases Used in Training
rmses = []
for i, case in enumerate(list(train.keys())[200:210]):
    print("~"*50)
    print(f"Predicting case {case}")
    rnn_dat = create_rnn_data2(train[case], params)
    m = rnn.predict(rnn_dat["X"])
    test_ind = rnn_dat['test_ind']
    rmses.append(rmse(m[test_ind:], rnn_dat['y_test'].flatten()))

In [ ]:
rmses

In [ ]:
pd.DataFrame({'Case': list(train.keys())[200:210], 'RMSE': rmses}).style.hide(axis="index")

In [ ]:
# Predict New Locations
rmses = []
for i, case in enumerate(list(train.keys())[150:180]):
    print("~"*50)
    print(f"Predicting case {case}")
    rnn_dat = create_rnn_data2(train[case], params)
    m = rnn.predict(rnn_dat["X"])
    test_ind = rnn_dat['test_ind']
    rmses.append(rmse(m[test_ind:], rnn_dat['y_test'].flatten()))

df = pd.DataFrame({'Case': list(train.keys())[150:180], 'RMSE': rmses})

In [ ]:
df[0:5].style.hide(axis="index")

In [ ]:
df.RMSE.mean()

In [ ]:
plt.hist(df.RMSE)